In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math

# Read Y data
gdp_per_capita = pd.read_csv("GDPperCapita/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_19346.csv", # GDP per capita (current US$) from World Bank
    skiprows=4,      # skip lines 0,1,2,3
    header=0, )
emissions = pd.read_csv("Emissions/API_EN.GHG.ALL.LU.MT.CE.AR5_DS2_en_csv_v2_42151.csv", # Total greenhouse gas emissions including LULUCF (Mt CO2e) from World Bank
    skiprows=4,      # skip lines 0,1,2,3
    header=0, )


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
import requests

# ------------------------------------------------------------------
# 1 · Read Y data (levels, not growth)
# ------------------------------------------------------------------
gdp_per_capita = pd.read_csv(
    "GDPperCapita/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_19346.csv",
    skiprows=4
)
emissions = pd.read_csv(
    "Emissions/API_EN.GHG.ALL.LU.MT.CE.AR5_DS2_en_csv_v2_42151.csv",
    skiprows=4
)

# drop empty rows / columns
for df in (gdp_per_capita, emissions):
    df.dropna(axis="columns", how="all", inplace=True)
    df.dropna(axis="index",   how="all", inplace=True)

# ------------------------------------------------------------------
# 2 · Wide → long
# ------------------------------------------------------------------
yrs = [str(y) for y in range(2000, 2020)]        # 2000‑2019 inclusive

gdp_long = gdp_per_capita.melt(
    id_vars=["Country Code"],
    value_vars=yrs,
    var_name="year",
    value_name="gdp_pc"            # rename for brevity
)
emiss_long = emissions.melt(
    id_vars=["Country Code"],
    value_vars=yrs,
    var_name="year",
    value_name="emissions"
)

for df in (gdp_long, emiss_long):
    df["year"] = df["year"].astype(int)

# ------------------------------------------------------------------
# 3 · Merge & build elasticity‑style β
#     β = Δlog(E) / Δlog(G)
# ------------------------------------------------------------------
panel = (
    emiss_long.merge(gdp_long, on=["Country Code", "year"], how="inner")
              .rename(columns={"Country Code": "country"})
              .sort_values(["country", "year"])
)

# log‑level differences (handles negative Δ because levels remain positive)
panel["dlogE"] = np.log(panel["emissions"]) - np.log(panel.groupby("country")["emissions"].shift(1))
panel["dlogG"] = np.log(panel["gdp_pc"])   - np.log(panel.groupby("country")["gdp_pc"].shift(1))

# elasticity: percent change in E divided by percent change in GDP
panel["beta"] = panel["dlogE"] / panel["dlogG"]

# drop rows with missing or undefined values (incl. first year per country)
panel = panel.dropna(subset=["beta"])

# optional 3‑year centred smoothing to reduce noise
panel["beta"] = (
    panel.groupby("country")["beta"]
         .transform(lambda s: s.rolling(window=3, center=True, min_periods=2).mean())
)
panel = panel.dropna(subset=["beta"])

# ------------------------------------------------------------------
# 4 · Keep only developing countries
# ------------------------------------------------------------------
def wb_income_iso3(level):
    url = f"https://api.worldbank.org/v2/incomeLevel/{level}/country?format=json&per_page=500"
    return [c["id"] for c in requests.get(url, timeout=30).json()[1]]

developing = set(wb_income_iso3("LIC") +
                 wb_income_iso3("LMC") +
                 wb_income_iso3("UMC"))

panel = panel[panel["country"].isin(developing)]

print(panel.head())
print(panel[["country", "year", "beta"]].describe())


c:\Users\winsb\OneDrive\Desktop\ISLP\.conda\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\winsb\OneDrive\Desktop\ISLP\.conda\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


     country  year  emissions      gdp_pc     dlogE     dlogG      beta
268      AFG  2001    22.7525  138.706822 -0.072434 -0.232029  0.364607
534      AFG  2002    25.3030  178.954088  0.106248  0.254767  0.314671
800      AFG  2003    25.8831  198.871116  0.022667  0.105528  0.205929
1066     AFG  2004    25.8435  221.763654 -0.001531  0.108955  0.092671
1332     AFG  2005    26.1174  254.184249  0.010543  0.136447  0.053587
              year         beta
count  1780.000000  1780.000000
mean   2010.058427     2.132897
std       5.465612    22.189317
min    2001.000000   -39.994441
25%    2005.000000    -0.107964
50%    2010.000000     0.203076
75%    2015.000000     0.805412
max    2019.000000   599.419688


In [3]:
renewable_energy = pd.read_csv("RenewableEnergyConsumption/API_EG.FEC.RNEW.ZS_DS2_en_csv_v2_21029.csv", # Renewable Energy Consumption from World Bank
                               skiprows=4,      # skip lines 0,1,2,3
                               header=0, )
fossil_fuel_consump = pd.read_csv("FossilFuelEnergyConsumption/API_EG.USE.COMM.FO.ZS_DS2_en_csv_v2_20914.csv", # Renewable Energy Consumption from World Bank
                                  skiprows=4,      # skip lines 0,1,2,3
                                  header=0, )
industry_value_added = pd.read_csv("IndustryValueAdded/API_NV.IND.TOTL.ZS_DS2_en_csv_v2_19918.csv", # Renewable Energy Consumption from World Bank
                                   skiprows=4,      # skip lines 0,1,2,3
                                   header=0, )
energy_intensity = pd.read_csv("EnergyIntensityPrimaryEnergy/API_EG.EGY.PRIM.PP.KD_DS2_en_csv_v2_23919.csv", 
                               skiprows=4, header=0,)
energy_imports = pd.read_csv("EnergyImports/API_EG.IMP.CONS.ZS_DS2_en_csv_v2_19399.csv",
                             skiprows=4, header=0,)
                             # ------------------------------------------------------------------
# 6 · melt predictors to long, merge, and run panel regression
# ------------------------------------------------------------------
# 6 · Melt predictors into long and merge with panel
# ------------------------------------------------------------------
predictors = {
    "renew_pct":        renewable_energy,
    "fossil_pct":       fossil_fuel_consump,
    "ind_va":           industry_value_added,
    "energy_intensity": energy_intensity,
    "energy_imports":   energy_imports
}

pred_long = None
for name, df in predictors.items():
    long = df.melt(
        id_vars=["Country Code"], value_vars=yrs,
        var_name="year", value_name=name
    )
    long["year"] = long["year"].astype(int)
    long = long.rename(columns={"Country Code": "country"})
    pred_long = long if pred_long is None else pred_long.merge(long, on=["country","year"], how="inner")

panel_reg = panel.merge(pred_long, on=["country","year"], how="inner").dropna()

# ------------------------------------------------------------------
# 7 · Run fixed‑effects panel regression
# ------------------------------------------------------------------
from linearmodels.panel import PanelOLS

data = panel_reg.set_index(["country","year"])

model = PanelOLS.from_formula(
    "beta ~ renew_pct + fossil_pct + ind_va + energy_intensity + energy_imports "
    "+ EntityEffects + TimeEffects",
    data=data
)
results = model.fit(cov_type="clustered", cluster_entity=True, cluster_time=True)

print(results.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   beta   R-squared:                        0.0053
Estimator:                   PanelOLS   R-squared (Between):             -5.1062
No. Observations:                1222   R-squared (Within):              -0.0082
Date:                Wed, May 07 2025   R-squared (Overall):             -1.3079
Time:                        22:56:09   Log-likelihood                   -5518.7
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.1865
Entities:                          76   P-value                           0.3136
Avg Obs:                       16.079   Distribution:                  F(5,1123)
Min Obs:                       1.0000                                           
Max Obs:                       19.000   F-statistic (robust):             0.5167
                            

In [4]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
from sklearn.preprocessing import StandardScaler

# 1 · build full panel (assumes `panel` with columns country, year, beta exists)
yrs = [str(y) for y in range(1990, 2020)]
files = {
    "renew_pct":        "RenewableEnergyConsumption/API_EG.FEC.RNEW.ZS_DS2_en_csv_v2_21029.csv",
    "fossil_pct":       "FossilFuelEnergyConsumption/API_EG.USE.COMM.FO.ZS_DS2_en_csv_v2_20914.csv",
    "ind_va":           "IndustryValueAdded/API_NV.IND.TOTL.ZS_DS2_en_csv_v2_19918.csv",
    "energy_intensity": "EnergyIntensityPrimaryEnergy/API_EG.EGY.PRIM.PP.KD_DS2_en_csv_v2_23919.csv",
    "energy_imports":   "EnergyImports/API_EG.IMP.CONS.ZS_DS2_en_csv_v2_19399.csv"
}

pred = None
for col, path in files.items():
    df = pd.read_csv(path, skiprows=4)
    long = df.melt(id_vars=["Country Code"], value_vars=yrs,
                   var_name="year", value_name=col)
    long["year"] = long["year"].astype(int)
    long = long.rename(columns={"Country Code": "country"})
    pred = long if pred is None else pred.merge(long, on=["country", "year"], how="inner")

panel_full = panel.merge(pred, on=["country", "year"], how="inner").dropna()

# 2 · lags, interaction, standardisation
for v in ["renew_pct", "fossil_pct", "ind_va", "energy_intensity", "energy_imports"]:
    panel_full[f"{v}_lag1"] = panel_full.groupby("country")[v].shift(1)

panel_full["ind_x_renew"] = panel_full["ind_va"] * panel_full["renew_pct"]
panel_full = panel_full.dropna()

zcols = [c for c in panel_full.columns if c not in ["country", "year", "beta"]]
panel_full[zcols] = StandardScaler().fit_transform(panel_full[zcols])

# 3 · two‑way fixed‑effects regression with Driscoll–Kraay (cov_type="kernel")
data = panel_full.set_index(["country", "year"])
formula = (
    "beta ~ renew_pct + renew_pct_lag1 + fossil_pct + fossil_pct_lag1 "
    "+ ind_va + ind_va_lag1 + energy_intensity + energy_intensity_lag1 "
    "+ energy_imports + energy_imports_lag1 + ind_x_renew "
    "+ EntityEffects + TimeEffects"
)

results = PanelOLS.from_formula(formula, data=data).fit(
    cov_type="kernel",        # Driscoll–Kraay HAC estimator
    kernel="bartlett",        # default kernel
    bandwidth=None            # automatic bandwidth selection
)

print(results.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   beta   R-squared:                        0.0057
Estimator:                   PanelOLS   R-squared (Between):             -0.3204
No. Observations:                1146   R-squared (Within):              -0.0038
Date:                Wed, May 07 2025   R-squared (Overall):             -0.0943
Time:                        22:56:10   Log-likelihood                   -4843.6
Cov. Estimator:        Driscoll-Kraay                                           
                                        F-statistic:                      0.5418
Entities:                          74   P-value                           0.8755
Avg Obs:                       15.486   Distribution:                 F(11,1044)
Min Obs:                       1.0000                                           
Max Obs:                       18.000   F-statistic (robust):             1.9137
                            